In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
from datetime import timedelta
import re
import shutil

import glob
%matplotlib inline

import matplotlib as mpl
mpl.use('agg')

# import personal modules
from plotter import plot_mclimate_forecast
import mclimate_funcs as mclim_func
from cw3e_tools import load_GFS_datasets, load_GEFS_datasets
from build_html_table import create_html_table

In [2]:
%%time

varname = 'ivt'
model = 'GEFS' ## 'GEFS' or 'GFS'

if model == 'GFS':
    ## using GFS data
    s = load_GFS_datasets(varname, fdate='2024072212') ## need to set date to what I have copied to personal dir
    forecast = s.calc_vars()

if model == 'GEFS':
    ## using GEFS data
    s = load_GEFS_datasets(varname)
    forecast = s.calc_vars()

## get month and date from the intialization date
ts = pd.to_datetime(forecast.init_date.values, format="%Y%m%d%H")
mon = ts.strftime('%m')
day = ts.strftime('%d')
print(mon, day)

## load mclimate data
mclimate = mclim_func.load_mclimate(mon, day, varname)

if model == 'GEFS':
    ## regrid/interpolate data to all have same grid size
    regrid_lats = forecast.lat
    regrid_lons = forecast.lon
    mclimate = mclimate.interp(lon=regrid_lons, lat=regrid_lats)
    
## compare the mclimate to the reforecast
ds = mclim_func.compare_mclimate_to_forecast(forecast, mclimate, varname)

step_lst = ds.step.values

for i, step in enumerate(step_lst):
    print(step)
    plot_mclimate_forecast(ds, forecast, step=step, varname='ivt')

/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'gini' loading failed:
Struct() takes at most 1 argument (3 given)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


08 14
6.0
12.0
18.0
24.0
30.0
36.0
42.0
48.0
54.0
60.0
66.0
72.0
78.0
84.0
90.0
96.0
102.0
108.0
114.0
120.0
126.0
132.0
138.0
144.0
150.0
156.0
162.0
168.0
CPU times: user 43.9 s, sys: 20 s, total: 1min 3s
Wall time: 34.5 s


In [3]:
varname = 'freezing_level'
date_string = ts.strftime('%Y%m%d%H')

if model == 'GFS':
    ## using GFS data
    date_string = ts.strftime('%Y%m%d%H')
    ## match the date of the IVT data
    s = load_GFS_datasets(varname, fdate=date_string) 
    forecast = s.calc_vars()

if model == 'GEFS':
    ## using GEFS data
    s = load_GEFS_datasets(varname, fdate=date_string)
    forecast = s.calc_vars()

## get month and date from the intialization date
ts = pd.to_datetime(forecast.init_date.values, format="%Y%m%d%H")
mon = ts.strftime('%m')
day = ts.strftime('%d')
print(mon, day)

## load mclimate data
mclimate = mclim_func.load_mclimate(mon, day, 'freezing_level')

if model == 'GEFS':
    ## regrid/interpolate data to all have same grid size
    regrid_lats = forecast.lat
    regrid_lons = forecast.lon
    mclimate = mclimate.interp(lon=regrid_lons, lat=regrid_lats)

## compare the mclimate to the reforecast
ds1 = mclim_func.compare_mclimate_to_forecast(forecast, mclimate, 'freezing_level')

step_lst = ds1.step.values

for i, step in enumerate(step_lst):
    print(step)
    plot_mclimate_forecast(ds1, forecast, step=step, varname='freezing_level')

08 14
6
12
18
24
30
36
42
48
54
60
66
72
78
84
90
96
102
108
114
120
126
132
138
144
150
156
162
168


In [4]:
%%time
ds = ds.rename({'mclimate': 'IVT'})
ds1 = ds1.rename({'mclimate': 'freezing_level'})
ds2 = xr.merge([ds, ds1])

ext_SEAK = [-141., -130., 54.5, 60.] # extent of SEAK 
df = create_html_table(ds2, ext_SEAK)

df



CPU times: user 58.9 ms, sys: 5.1 ms, total: 64 ms
Wall time: 65.6 ms


In [5]:
## write to text file
html = df.to_html(index=False, formatters={'Hour': lambda x: '<b>' + x + '</b>'}, escape=False)

# write html to file
text_file = open("out/table_test.html", "w")
text_file.write(html)
text_file.close()

In [6]:
# # Define two pairs of colors (dark and light green/yellow)

# from itertools import cycle

# colors = [(245./255., 240./255., 230./255.), (182./255., 177./255., 169./255.)]  # green, yellow
# color_cycle = cycle(
#     [
#         {
#             k: (int(c[0] * 255), int(c[1] * 255), int(c[2] * 255), a)
#             for k, a in enumerate([1, 0.25])
#         }
#         for c in colors
#     ]
# )

# # Define color for each row

# bg_colors = []

# for i in t.index.get_level_values(0).unique():
#     color = next(color_cycle)
#     row_color = cycle(
#         [
#             {
#                 "props": [("background-color", f"rgba{color[0]}")],
#             },
#             {
#                 "props": [("background-color", f"rgba{color[1]}")],
#             },
#         ]
#     )
#     for _ in range(t.loc[(i,), :].shape[0]):
#         bg_colors.append(next(row_color))
        
# # Style dataframe

# css = [{"selector": f".row{i}"} | color for i, color in enumerate(bg_colors)]

# t.style.set_table_styles(css)

In [7]:
# def get_lat_lons_from_txt_file(textpts_fname):
#     ## read text file with points
#     df = pd.read_csv(textpts_fname, header=None, sep=' ', names=['latitude', 'longitude'], engine='python')
#     df['longitude'] = df['longitude']*-1
#     df = df
#     lons = df['longitude'].values
#     lats = df['latitude'].values

#     x = xr.DataArray(lons, dims=['location'])
#     y = xr.DataArray(lats, dims=['location'])

#     return x, y
# ## select transect points
# x, y = get_lat_lons_from_txt_file('../data/latlon_coast.txt')
# mclimate = mclimate.sel(lon=x, lat=y, method='nearest')
# forecast = forecast.sel(lon=x, lat=y, method='nearest')
# ## apply below loop to transect points only